Name: Rahul

Roll number: 22103045

Kaggle ID : rahultaank

# **ASSIGNMENT 2**

# CRACK DETECTION IMAGE CLASSIFICATION USING CNN MODEL

# importing libraries

In [21]:
#  !pip install tensorflow



In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report
import os
import cv2
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential

from glob import glob


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

# TRAINING IMAGE PATHFILE

In [ ]:
cracked_dir = r'K:\IITK\SEMESTER 2\MACHINE LEARNING\ASSIGNMENT 2\train\cracked'
uncracked_dir = r'K:\IITK\SEMESTER 2\MACHINE LEARNING\ASSIGNMENT 2\train\uncracked'

# DATAFRAMES

In [ ]:
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [ ]:
cracked_df = generate_df(cracked_dir, label="cracked")
uncracked_df = generate_df(uncracked_dir, label="uncracked")

all_df = pd.concat([cracked_df, uncracked_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
all_df

In [ ]:
from pathlib import Path

def generate_df(image_dir, label):
    image_dir_path = Path(image_dir)  # Convert the image_dir string to a Path object
    filepaths = pd.Series(list(image_dir_path.glob('*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

# Assuming cracked_dir and uncracked_dir are your image directory paths as strings
cracked_df = generate_df(cracked_dir, label="cracked")
uncracked_df = generate_df(uncracked_dir, label="uncracked")

all_df = pd.concat([cracked_df, uncracked_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
all_df


# Pie chart representing percentage of data classification

In [ ]:
freq = all_df['Label'].value_counts()
print(freq)
freq.plot(kind='pie', figsize=(3,3), title='Cracks', autopct='%1.2f%%',  shadow = False)

In [ ]:
train_df, test_df = train_test_split(all_df.sample(14968, random_state=1),train_size=0.9,shuffle=True,random_state=1)

In [ ]:
train_df

In [ ]:
test_df

# Image Data Loading

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,shear_range = 0.2,
                                   zoom_range = 0.05,
                                   horizontal_flip = True,
                                   vertical_flip   = True,
    validation_split=0.1
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_data = train_gen.flow_from_dataframe(
    train_df,
x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42,
    subset='training')

val_data= train_gen.flow_from_dataframe(
    train_df,
x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42,
    subset='validation'
)


test_data = train_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(256,256),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)

# LOADING MODEL (InceptionResNetV2)

In [ ]:
# Import the InceptionV3 library as shown below and add preprocessing layer to the front of InceptionV3
# Here we will be using imagenet weights
IMAGE_SIZE= [256, 256]

inception = tf.keras.applications.InceptionResNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

inception.trainable = True

set_trainable = False

for layer in inception.layers:
    if layer.name == 'conv2d_90':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
# our layers - you can add more if you want

x = Flatten()(inception.output)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inception.inputs, outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

# Training data

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

# Graph plotting

In [ ]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss Over Time"
)

fig.show()

# Results

In [ ]:
def evaluate_model(model, test_data):
    
    results = model.evaluate(test_data, verbose=0)
    loss = results[0]
    acc = results[1]
    
    print("    Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))
    
    y_pred = np.squeeze((model.predict(test_data) >= 0.3931).astype(np.int))
    cm = confusion_matrix(test_data.labels, y_pred)
    clr = classification_report(test_data.labels, y_pred, target_names=["uncracked", "cracked"])
    
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
    plt.xticks(ticks=np.arange(2) + 0.5, labels=["uncracked", "cracked"])
    plt.yticks(ticks=np.arange(2) + 0.5, labels=["uncracked", "cracked"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
    
    print("Classification Report:\n----------------------\n", clr)

In [ ]:
evaluate_model(model, test_data)

In [ ]:
#ROC

y_preds = model.predict(test_data).ravel()
#y_pred = np.squeeze((model.predict(test_data) >= 0.3931).astype(np.int))
fpr, tpr, thresholds = roc_curve(test_data.labels, y_preds)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'y--')
plt.plot(fpr, tpr, marker='.')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()

In [ ]:
from sklearn.metrics import auc
i = np.arange(len(tpr)) 
roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'thresholds' : pd.Series(thresholds, index=i)})
ideal_roc_thresh = roc.iloc[(roc.tf-0).abs().argsort()[:1]]  #Locate the point where the value is close to 0
print("Ideal threshold is: ", ideal_roc_thresh['thresholds']) 
auc_value = auc(fpr, tpr)
print("Area under curve, AUC = ", auc_value)


# Predictions

In [ ]:
test_path = '/kaggle/input/crack-detection-image-classification-2023/test'

In [ ]:
test_filenames = os.listdir(test_path)

In [ ]:
a = train_data.class_indices
class_names = list(a.keys())

In [ ]:
test_predictions = np.array([])

for img in os.listdir(test_path):
    image_path = os.path.join(test_path,img)
    image = tf.keras.utils.load_img(image_path, target_size = (256,256))
    image = tf.keras.utils.img_to_array(image)
    image = np.array([image])
    image = image/255
    y_pred = model.predict(image)
    y_pred_label = np.squeeze((y_pred >= 0.5861).astype(int))
    test_predictions = np.append(test_predictions, class_names[y_pred_label])
#     break
# print(test_predictions[0])


# Output file

In [ ]:

# Create a DataFrame with the filename and predicted values
results_df = pd.DataFrame({
    "filename": test_filenames,
    "class": test_predictions
})

# Save the DataFrame to a CSV file
results_df.to_csv("submission.csv", index=False)